In [128]:
from huggingface_hub import notebook_login
notebook_login()

squad_v2 = False
model_checkpoint = "distilbert-base-uncased-distilled-squad"
batch_size = 16


In [129]:
import transformers
print(transformers.__version__)

from transformers.utils import send_example_telemetry
send_example_telemetry("question_answering_notebook", framework="pytorch")

# datasets transformers huggingface_hub ipywidgets

4.30.2


In [1]:
from datasets import load_dataset, load_metric

from datasets import load_dataset

# datasets = load_dataset("squad_v2" if squad_v2 else "squad")
datasets = load_dataset("csv", data_files="question_answer.csv")
dataset_train = datasets['train'].train_test_split(test_size=0.2, seed=42)
datasets['validation'] = dataset_train['test']
datasets['train'] = dataset_train['train']


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/mvinn/.cache/huggingface/datasets/csv/default-d03773f789d1f381/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 602
    })
})


In [131]:
import json

# transform the answers from string to dict
datasets['train'] = datasets['train'].map(lambda example: { 
    **example, 
    'answers': json.loads(example['answers'].replace("'", '"')),
})
datasets['validation'] = datasets['validation'].map(lambda example: { 
    **example, 
    'answers': json.loads(example['answers'].replace("'", '"')),
})

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [132]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [133]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [134]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

for i, example in enumerate(datasets["train"]):
  if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length:
    break
example = datasets["train"][i]

In [135]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

207

In [136]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [137]:
[len(x) for x in tokenized_example["input_ids"]]

[207]

In [138]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] whats the type of porygon? [SEP] porygon its a normal first generation pokemon that porygon has become notorious for being featured in an episode of the anime that caused widespread epileptic seizures in japanese viewers. roughly made about 20 years ago, it's entirely made of programming code. it can enter cyberspace at will. people theorized it could go into space, though no one has managed to do so. it has no heartbeat and doesn't need to eat, though it can eat if food is given. because of this, people are eager to try it in any environment. it's copy - protected so it cannot be duplicated. using the most advanced technology at the time, scientists have made the first man made pokemon, dreaming of exploring space. nowadays, it's actually very useful in cyberspace, as it goes around making sure no suspicious data exists. it is one of many human - designed pokemon. it gained an evolution in generation ii, porygon2. [SEP]


In [139]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (4, 5), (6, 9), (10, 14), (15, 17), (18, 21), (21, 22), (22, 25), (25, 26), (0, 0), (0, 3), (3, 4), (4, 7), (8, 11), (12, 13), (14, 20), (21, 26), (27, 37), (38, 45), (46, 50), (51, 54), (54, 55), (55, 58), (59, 62), (63, 69), (70, 79), (80, 83), (84, 89), (90, 98), (99, 101), (102, 104), (105, 112), (113, 115), (116, 119), (120, 125), (126, 130), (131, 137), (138, 148), (149, 151), (151, 154), (154, 158), (159, 167), (168, 170), (171, 179), (180, 187), (187, 188), (189, 196), (197, 201), (202, 207), (208, 210), (211, 216), (217, 220), (220, 221), (222, 224), (224, 225), (225, 226), (227, 235), (236, 240), (241, 243), (244, 255), (256, 260), (260, 261), (262, 264), (265, 268), (269, 274), (275, 280), (280, 285), (286, 288), (289, 293), (293, 294), (295, 301), (302, 306), (306, 311), (312, 314), (315, 320), (321, 323), (324, 328), (329, 334), (334, 335), (336, 342), (343, 345), (346, 349), (350, 353), (354, 361), (362, 364), (365, 367), (368, 370), (370, 371), (372, 374

In [140]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

what What


In [141]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None]


In [142]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

16 16


In [143]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

normal
Normal


In [144]:
pad_on_right = tokenizer.padding_side == "right"

In [145]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [146]:
features = prepare_train_features(datasets['train'][:5])

In [147]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


Map:   0%|          | 0/121 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

In [148]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [149]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [150]:
from transformers import default_data_collator

data_collator = default_data_collator

In [151]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model("pokemon-gpt-trained")

c:\repositorios\projeto-paa\distilbert-base-uncased-distilled-squad-finetuned-squad is already a clone of https://huggingface.co/M4ycon/distilbert-base-uncased-distilled-squad-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.1145409494638443, 'eval_runtime': 12.152, 'eval_samples_per_second': 2.551, 'eval_steps_per_second': 0.165, 'epoch': 1.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.08186446875333786, 'eval_runtime': 11.816, 'eval_samples_per_second': 2.624, 'eval_steps_per_second': 0.169, 'epoch': 2.0}


  0%|          | 0/2 [00:00<?, ?it/s]

{'eval_loss': 0.0654781386256218, 'eval_runtime': 11.9466, 'eval_samples_per_second': 2.595, 'eval_steps_per_second': 0.167, 'epoch': 3.0}
{'train_runtime': 404.6076, 'train_samples_per_second': 0.897, 'train_steps_per_second': 0.059, 'train_loss': 0.3542122046152751, 'epoch': 3.0}


Upload file training_args.bin: 100%|##########| 3.93k/3.93k [00:00<?, ?B/s]

To https://huggingface.co/M4ycon/distilbert-base-uncased-distilled-squad-finetuned-squad
   dcd341b..3064f1c  main -> main



In [152]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [153]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [154]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([16, 16, 16, 16, 16, 12, 14, 14, 14, 14, 16, 14, 14, 14, 16, 16]),
 tensor([16, 16, 16, 16, 16, 12, 14, 14, 14, 14, 16, 14, 14, 14, 16, 16]))

In [155]:
n_best_size = 20
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [156]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [157]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)
raw_predictions = trainer.predict(validation_features)
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [158]:
raw_predictions = trainer.predict(validation_features)

  0%|          | 0/2 [00:00<?, ?it/s]

In [159]:
max_answer_length = 30
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 17.619402, 'text': 'Water'},
 {'score': 8.230536, 'text': 'Gyarados its a Water'},
 {'score': 7.132507, 'text': 'Water first generation'},
 {'score': 6.257944, 'text': 'Water first generation Pokemon'},
 {'score': 5.771485, 'text': 'Water first'},
 {'score': 5.7145643, 'text': 'a Water'},
 {'score': 5.320012,
  'text': 'Water first generation Pokemon that In the beta'},
 {'score': 5.283752,
  'text': 'Water first generation Pokemon that In the beta versions for Pokémon Red and Blue, Gyarados was originally named "Skullkraken".'},
 {'score': 4.9621506,
  'text': 'Water first generation Pokemon that In the beta versions for Pokémon Red and Blue, Gyarados'},
 {'score': 4.9185905,
  'text': 'Water first generation Pokemon that In the beta versions for Pokémon Red and Blue, Gyarados was originally named "Skullkraken". Gyarados'},
 {'score': 4.9027376, 'text': 'its a Water'},
 {'score': 4.8370323,
  'text': 'Water first generation Pokemon that In the beta versions for Pokémon Red 

In [160]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [161]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [162]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 31 example predictions split into 31 features.


  0%|          | 0/31 [00:00<?, ?it/s]

In [163]:
metric = load_metric("squad_v2" if squad_v2 else "squad")

if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]
references = [{"id": ex["id"], "answers": ex["answers"]} for ex in datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)

{'exact_match': 96.7741935483871, 'f1': 96.7741935483871}

In [164]:
trainer.push_to_hub()

In [165]:
from transformers import AutoModelForQuestionAnswering, pipeline

tokenizer = AutoTokenizer.from_pretrained("M4ycon/distilbert-base-uncased-finetuned-squad")
model = AutoModelForQuestionAnswering.from_pretrained("M4ycon/distilbert-base-uncased-finetuned-squad")


In [166]:
question = "type Charmander" # electric
# question = "Whats the type of Charmander?" # psychic
# context = "Pokémon is set in the fictional Pokémon universe. There are numerous regions that have appeared in the various media of the Pokémon franchise. There are 9 main series regions set in the main series games: Kanto, Johto, Hoenn, Sinnoh/Hisui, Unova, Kalos, Alola, Galar, and Paldea. Each of the nine generations of the main series releases focuses on a new region. Every region consists of several cities and towns that the player must explore in order to overcome many waiting challenges, such as Gyms, Contests and villainous teams. At different locations within each region, the player can find different types of Pokémon, as well as helpful items and characters. Different regions are not accessible from one another at all within a single game, only with the exception of Kanto and Johto being linked together in Pokémon Gold, Silver, Crystal, HeartGold and SoulSilver versions. There are also regions set in spinoff games and two islands in the Pokémon anime (Orange Islands and Decolore Islands), all still set within the same fictional universe."
# context = "Squirtle its a Water first generation Pokemon that It is a playable character in Super Smash Bros. Brawl and Super Smash Bros. Ultimate as a part of the \"Pokémon Trainer\" fighter. Its English name is a portmanteau of \"squirt\" and \"turtle\". It is one of the Kanto starter Pokémon.[9] When born, its back swells into a shell which after some time will become resilient, and it takes time for the shell to harden. When it feels threatened, it tucks in its limbs and hides in its shell. Squirtle's shell is not merely used for protection as the shell's rounded shape and the grooves on its surface help minimize water resistance, enabling this Pokémon to swim at high speeds. It can spray a powerful foam from its mouth by retracting its long neck and vigorously shooting water out."
context = open('./big-context.txt', mode="r", encoding="utf-8").read()

question = question.lower().strip()
context = context.lower().strip()

In [167]:

question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
print('Resposta:', question_answerer(question=question, context=context)['answer'])

Resposta: electric
